In [1]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

In [42]:
start_date = datetime.now() - timedelta(days=120)
start_date_str = start_date.strftime('%Y%m%d')
stock_code='601919'
df = ak.stock_zh_a_hist(
    symbol=stock_code, start_date=start_date_str, adjust='')
df['pct_change'] = df['收盘'].pct_change(periods =1) #计算百分比变化
df['MA_60'] = df['收盘'].rolling(window=60).mean()
df['Above_MA_60'] = (df['收盘'] > df['MA_60']).astype(int)
print(df[["日期",  "开盘", "最高" , "最低" ,  "收盘" ,"MA_60", "pct_change","Above_MA_60"]])


            日期     开盘     最高     最低     收盘      MA_60  pct_change  Above_MA_60
0   2024-02-29  10.41  10.52  10.38  10.50        NaN         NaN            0
1   2024-03-01  10.45  10.49  10.38  10.42        NaN   -0.007619            0
2   2024-03-04  10.38  10.50  10.31  10.44        NaN    0.001919            0
3   2024-03-05  10.44  10.51  10.40  10.48        NaN    0.003831            0
4   2024-03-06  10.48  10.49  10.41  10.43        NaN   -0.004771            0
..         ...    ...    ...    ...    ...        ...         ...          ...
76  2024-06-24  15.90  15.99  15.60  15.63  13.129667   -0.026168            1
77  2024-06-25  15.51  15.82  15.41  15.52  13.218000   -0.007038            1
78  2024-06-26  15.40  15.55  15.14  15.49  13.306167   -0.001933            1
79  2024-06-27  15.49  15.69  14.90  15.02  13.385000   -0.030342            1
80  2024-06-28  15.00  15.63  14.95  15.49  13.470167    0.031292            1

[81 rows x 8 columns]


In [48]:
recent_days = df['pct_change'].tail(13)
max_pct_change_idx = recent_days.idxmax()

max_pct_change_date = df.loc[max_pct_change_idx, '日期']
max_pct_change_date_low = df.loc[max_pct_change_idx, '最低']
max_pct_change_max = df.loc[max_pct_change_idx, 'pct_change']
print(recent_days)
print(max_pct_change_idx,max_pct_change_max)
# 从那一天开始，找到所有后续交易日的最低价
# 使用布尔索引来选择那一天之后的所有日期
mask = df['日期'] >= max_pct_change_date
min_low_after = df.loc[mask, '最低'].min()
print(min_low_after)
# 计算交易日数量
trading_days_after = df.loc[mask]
trading_days_count = trading_days_after.shape[0]
print(min_low_after,trading_days_count)
# 最近一次大涨后，接下来的交易日是否跌穿过大涨当日的最低价
Above_MA_60 = df.loc[df.index[-1], 'Above_MA_60']
print (Above_MA_60,trading_days_count, min_low_after , max_pct_change_date_low, min_low_after > max_pct_change_date_low)

above_support_after_rally = Above_MA_60 == 1 and trading_days_count >3 and trading_days_count<10 and min_low_after > max_pct_change_date_low
print(above_support_after_rally)

68   -0.012937
69   -0.015727
70    0.012650
71   -0.012492
72    0.036618
73    0.000642
74    0.028883
75    0.001248
76   -0.026168
77   -0.007038
78   -0.001933
79   -0.030342
80    0.031292
Name: pct_change, dtype: float64
72 0.03661784287616521
14.9
14.9 9
1 9 14.9 15.02 False
False


In [ ]:
# 找到那一天的日期和最低价
max_pct_change_date = df.loc[max_pct_change_idx, 'Date']
max_pct_change_low = df.loc[max_pct_change_idx, 'Close']

In [49]:
(6.27 - 5.75) / 5.75

0.09043478260869557

In [55]:
query = f"现价大于5元的今日涨停封成比前30的主板股票"
df = pywencai.get(query=query, query_type="stock")
print(df.columns)
print(df)

Index(['股票代码', '股票简称', '最新涨跌幅', '收盘价:不复权[20240701]', '上市板块',
       '涨停封单量占成交量比[20240701]', '涨停封单量占成交量比排名[20240701]', '开盘价:不复权[20240701]',
       '最高价:不复权[20240701]', '最低价:不复权[20240701]', '注册地址', '经营范围', 'market_code',
       'code'],
      dtype='object')
         股票代码  股票简称   最新涨跌幅 收盘价:不复权[20240701] 上市板块  涨停封单量占成交量比[20240701]  \
0   002857.SZ  三晖电气  10.007             15.17   主板            396.293338   
1   002645.SZ  华宏科技  10.063              7.00   主板            101.231101   
2   002657.SZ  中科金财  10.035             12.50   主板             66.070182   
3   603829.SH  洛凯股份   9.975             13.01   主板             49.716592   
4   003042.SZ  中农联合   9.992             12.88   主板             28.984370   
5   002861.SZ  瀛通通讯  10.009             11.76   主板             23.356833   
6   002244.SZ  滨江集团  10.055              7.99   主板             20.753921   
7   600338.SH  西藏珠峰   9.957             10.16   主板             19.161079   
8   603171.SH  税友股份   9.985             29.74   主板         

In [61]:
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="000001", start_date="2024-07-01 09:30:00", end_date="2024-07-01 09:50:00", period="1", adjust="")
print(stock_zh_a_hist_min_em_df)

                     时间     开盘     收盘     最高     最低    成交量         成交额     最新价
0   2024-07-01 09:30:00  10.09  10.09  10.09  10.09  20349  20532141.0  10.090
1   2024-07-01 09:31:00  10.09  10.06  10.09  10.04  49671  49982496.0  10.071
2   2024-07-01 09:32:00  10.06  10.07  10.07  10.05  11393  11464577.0  10.070
3   2024-07-01 09:33:00  10.06  10.08  10.08  10.06  11443  11517422.0  10.069
4   2024-07-01 09:34:00  10.07  10.07  10.08  10.06  10381  10454128.0  10.069
5   2024-07-01 09:35:00  10.07  10.07  10.07  10.06   6528   6568637.0  10.069
6   2024-07-01 09:36:00  10.07  10.07  10.08  10.06  10682  10754580.0  10.069
7   2024-07-01 09:37:00  10.07  10.07  10.08  10.06   5341   5378077.0  10.069
8   2024-07-01 09:38:00  10.07  10.05  10.08  10.05  27939  28095108.0  10.066
9   2024-07-01 09:39:00  10.05  10.05  10.06  10.05   9340   9388781.0  10.066
10  2024-07-01 09:40:00  10.06  10.06  10.06  10.05   5189   5216797.0  10.065
11  2024-07-01 09:41:00  10.05  10.05  10.06  10.05 

In [66]:
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="300401", start_date="2024-07-01 09:15:00", end_date="2024-07-01 09:59:00", period="1", adjust="")
print(stock_zh_a_hist_min_em_df)

                     时间     开盘     收盘     最高     最低    成交量         成交额     最新价
0   2024-07-01 09:30:00  14.59  14.59  14.59  14.59  11324  16521716.0  14.590
1   2024-07-01 09:31:00  14.70  14.46  14.78  14.34  46518  67633902.0  14.549
2   2024-07-01 09:32:00  14.50  14.61  14.74  14.50  22830  33379216.0  14.570
3   2024-07-01 09:33:00  14.63  14.43  14.63  14.40  14272  20720487.0  14.562
4   2024-07-01 09:34:00  14.55  14.46  14.55  14.43  11881  17216280.0  14.554
5   2024-07-01 09:35:00  14.46  14.44  14.53  14.44   8227  11917462.0  14.549
6   2024-07-01 09:36:00  14.44  14.34  14.44  14.30  16885  24307019.0  14.529
7   2024-07-01 09:37:00  14.31  14.24  14.31  14.23  14035  20050225.0  14.506
8   2024-07-01 09:38:00  14.26  14.29  14.33  14.24   9496  13562082.0  14.492
9   2024-07-01 09:39:00  14.28  14.15  14.28  14.10  14864  21083729.0  14.465
10  2024-07-01 09:40:00  14.15  14.23  14.26  14.15   9068  12855125.0  14.451
11  2024-07-01 09:41:00  14.21  14.15  14.21  14.14 

In [2]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

In [3]:
df = ak.stock_zh_a_hist_pre_min_em(symbol='300397', start_time="09:00:00", end_time="09:31:00")
df

,时间,开盘,收盘,最高,最低,成交量,成交额,最新价
0,2024-07-05 09:15:00,8.90,8.90,8.90,8.90,0,0.0,8.900
1,2024-07-05 09:16:00,8.90,8.90,8.90,8.90,0,0.0,8.900
2,2024-07-05 09:17:00,8.86,8.86,8.86,8.86,0,0.0,8.900
3,2024-07-05 09:18:00,8.90,8.90,8.90,8.90,0,0.0,8.900
4,2024-07-05 09:19:00,8.90,8.90,8.90,8.90,0,0.0,8.900
5,2024-07-05 09:20:00,8.90,8.90,8.90,8.90,0,0.0,8.900
6,2024-07-05 09:21:00,8.90,8.90,8.90,8.90,0,0.0,8.900
7,2024-07-05 09:22:00,8.90,8.90,8.90,8.90,0,0.0,8.900
8,2024-07-05 09:23:00,8.90,8.90,8.90,8.90,0,0.0,8.900
9,2024-07-05 09:24:00,8.90,8.90,8.90,8.90,0,0.0,8.900


In [3]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>5
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1.001
  price_times =( df.iloc[-2]['最新价'] / df.iloc[-6]['最新价'] )>1
  # volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>=3
  # volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1
  # price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>=1.001
  # print(volume_times ,  df.iloc[-1]['最新价'] , df.iloc[-6]['最新价'], df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'])
  return volume_times and price_times and volume_times_32

df = ak.stock_zh_a_spot_em()
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['代码'].astype(str).str.startswith('4')]
df = df[~df['代码'].astype(str).str.startswith('8')]
df = df[~df['代码'].astype(str).str.startswith('68')]
df = df[~df['名称'].astype(str).str.startswith('N')]
df = df[~df['名称'].astype(str).str.startswith('*')]
df = df[~df['名称'].astype(str).str.startswith('ST')]
# 按成交额降序排序，选出开盘金额最大前N只股
df = df.sort_values(by='成交额', ascending=False)
df = df.head(300)
df['代码'] = df['代码'].astype(str).str[:6]

df['open_up'] = df['今开'] >0.8 * df['昨收']
df = df.where(df['open_up'], np.nan).dropna(
    subset=['open_up'])

# df['量比大于2'] = df['量比'] >= 1.2
# df = df.where(df['量比大于2'], np.nan).dropna(
#     subset=['量比大于2'])

# df['60日涨跌幅大于0'] = df['60日涨跌幅'] > 0
# df = df.where(df['60日涨跌幅大于0'], np.nan).dropna(
#     subset=['60日涨跌幅大于0'])

df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df = df.where(df['volume_up_and_price_up'], np.nan).dropna(
    subset=['volume_up_and_price_up'])
# print(df[['代码','名称','昨收','今开','最高','最低','最新价','涨跌幅','成交额','量比','振幅']])
# print(df[['代码','名称','昨收','今开','最新价','涨跌幅','成交额','量比','振幅']])
df["ok"] = df['最新价'] > df['今开']
print(df[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']])

          代码     名称     ok      昨收      今开     最新价   涨跌幅           成交额
36    601888   中国中免   True   61.81   62.03   67.98  9.98  4.405745e+09
2386  002475   立讯精密  False   39.09   39.09   38.89 -0.51  2.051170e+09
62    300458   全志科技   True   22.68   22.60   24.31  7.19  1.380605e+09
1012  000002  万  科Ａ   True    7.13    7.13    7.16  0.42  1.166482e+09
701   601766   中国中车   True    7.67    7.67    7.74  0.91  1.125058e+09
804   300476   胜宏科技   True   33.38   33.55   33.62  0.72  1.120671e+09
1169  002241   歌尔股份   True   19.53   19.52   19.58  0.26  1.080095e+09
793   000725   京东方Ａ   True    4.10    4.09    4.13  0.73  9.029732e+08
152   000049   德赛电池   True   24.89   24.80   26.03  4.58  8.294823e+08
818   002428   云南锗业   True   11.44   11.41   11.52  0.70  7.605476e+08
500   300346   南大光电   True   28.31   28.10   28.72  1.45  7.599902e+08
704   300433   蓝思科技   True   17.69   17.70   17.85  0.90  6.836532e+08
1257  600887   伊利股份   True   25.77   25.77   25.81  0.16  6.725615e+08
3866  

In [1]:
df_ok_not = df.where(df['ok'].astype(bool)== False, np.nan).dropna(
    subset=['ok'])
df_ok_not[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']]

NameError: name 'df' is not defined

In [215]:
df_ok = df.where(df['ok'], np.nan).dropna(
    subset=['ok'])
df_ok[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']]


,代码,名称,ok,昨收,今开,最新价,涨跌幅,成交额
113,601888,中国中免,True,61.81,62.03,64.42,4.22,8.958249e+08
90,000049,德赛电池,True,24.89,24.80,26.11,4.90,4.699459e+08
632,300476,胜宏科技,True,33.38,33.55,33.80,1.26,3.835218e+08
359,002428,云南锗业,True,11.44,11.41,11.66,1.92,2.283745e+08
1681,300433,蓝思科技,True,17.69,17.70,17.76,0.40,1.886699e+08
559,300458,全志科技,True,22.68,22.60,23.00,1.41,1.778627e+08
23,000007,全新好,True,5.21,5.30,5.73,9.98,1.348927e+08
69,603179,新泉股份,True,39.83,40.60,41.98,5.40,1.171225e+08
1726,603799,华友钴业,True,21.95,21.93,22.03,0.36,1.167467e+08
270,603099,长白山,True,22.94,23.09,23.48,2.35,1.136186e+08


In [9]:
import pywencai
import pandas as pd
import numpy as np
import akshare as ak
from datetime import datetime, timedelta

def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:32:00")
  volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>5
  volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1.001
  price_times =( df.iloc[-2]['最新价'] / df.iloc[-6]['最新价'] )>1
  # volume_times =( df.iloc[-2]['成交量'] / df.iloc[-6]['成交量'] )>=3
  # volume_times_32 =( df.iloc[-1]['成交量'] / df.iloc[-2]['成交量'] )>1
  # price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>=1.001
  # print(volume_times ,  df.iloc[-1]['最新价'] , df.iloc[-6]['最新价'], df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'])
  return volume_times and price_times and volume_times_32

df = ak.stock_zh_a_spot_em()
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['代码'].astype(str).str.startswith('4')]
df = df[~df['代码'].astype(str).str.startswith('8')]
df = df[~df['代码'].astype(str).str.startswith('68')]
df = df[~df['名称'].astype(str).str.startswith('N')]
df = df[~df['名称'].astype(str).str.startswith('*')]
df = df[~df['名称'].astype(str).str.startswith('ST')]
# 按成交额降序排序，选出开盘金额最大前N只股
df = df.sort_values(by='成交额', ascending=False)
df = df.head(300)
df['代码'] = df['代码'].astype(str).str[:6]

df['open_up'] = df['今开'] >0.8 * df['昨收']
df = df.where(df['open_up'], np.nan).dropna(
    subset=['open_up'])

df['60日涨跌幅大于0'] = df['60日涨跌幅'] > 0
df = df.where(df['60日涨跌幅大于0'], np.nan).dropna(
    subset=['60日涨跌幅大于0'])

df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df = df.where(df['volume_up_and_price_up'], np.nan).dropna(
    subset=['volume_up_and_price_up'])
# print(df[['代码','名称','昨收','今开','最高','最低','最新价','涨跌幅','成交额','量比','振幅']])
# print(df[['代码','名称','昨收','今开','最新价','涨跌幅','成交额','量比','振幅']])
df["ok"] = df['最新价'] > df['今开']
# print(df[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']])

df_ok = df.where(df['ok'], np.nan).dropna(
    subset=['ok'])
df_ok[['代码','名称','ok','昨收','今开','最新价','涨跌幅','成交额']]

,代码,名称,ok,昨收,今开,最新价,涨跌幅,成交额
160,000333,美的集团,True,63.03,63.20,64.50,2.33,1.046497e+09
26,300120,经纬辉开,True,6.04,6.00,6.55,8.44,4.844470e+08
176,600522,中天科技,True,15.17,15.19,15.50,2.18,4.364445e+08
62,600312,平高电气,True,18.60,18.60,19.46,4.62,4.025435e+08
194,000400,许继电气,True,32.66,32.52,33.31,1.99,3.701581e+08
3,300486,东杰智能,True,4.94,4.97,5.68,14.98,3.588358e+08
103,002351,漫步者,True,12.30,12.31,12.72,3.41,3.139371e+08
385,300386,飞天诚信,True,9.69,9.75,9.78,0.93,2.746150e+08
544,600919,江苏银行,True,7.54,7.54,7.58,0.53,2.543040e+08
0,300546,雄帝科技,True,11.43,11.31,13.72,20.03,2.189223e+08


In [15]:
query = f"个股热度前200名"
df = pywencai.get(query=query, query_type="stock")
# df.loc[df['股票代码'].astype(str).str.startswith('3'), 'max_pct'] = 20
# df.loc[~df['股票代码'].astype(str).str.startswith('3'), 'max_pct'] = 10
# df['max_pct'] = 20 * df['股票代码'].str.startswith('3')
df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)
df[['股票代码','max_pct']] 

,股票代码,max_pct
0,600518.SH,10
1,603350.SH,10
2,600297.SH,10
3,603920.SH,10
4,002979.SZ,10
...,...,...
95,000004.SZ,10
96,300098.SZ,20
97,603227.SH,10
98,300033.SZ,20


In [18]:
query = f"个股热度前500名"
df = pywencai.get(query=query, query_type="stock",loop=True)
df

,股票代码,股票简称,最新价,最新涨跌幅,个股热度[20240705],个股热度排名[20240705],market_code,code
0,600733.SH,北汽蓝谷,9.12,9.880,490809.5,1,17,600733
1,603350.SH,安乃达,43.55,-20.092,207564.0,2,17,603350
2,300391.SZ,长药控股,4.62,20.000,197674.0,3,33,300391
3,603501.SH,韦尔股份,97.68,-0.194,194370.0,4,17,603501
4,603285.SH,键邦股份,43.50,133.244,185194.0,5,17,603285
...,...,...,...,...,...,...,...,...
495,300185.SZ,通裕重工,1.81,5.848,8270.5,496,33,300185
496,002722.SZ,物产金轮,11.52,5.980,8269.5,497,33,002722
497,002223.SZ,鱼跃医疗,38.04,4.448,8257.0,498,33,002223
498,600053.SH,九鼎投资,13.38,1.904,8248.0,499,17,600053


In [17]:
def volume_up_and_price_up(symbol:str):
  df = ak.stock_zh_a_hist_pre_min_em(symbol=symbol, start_time="09:00:00", end_time="09:31:00")
  volume_times =( df.iloc[-1]['成交量'] / df.iloc[-6]['成交量'] )> 7
  volume_times_31 = df.iloc[-1]['成交量'] > 4000
  price_times =( df.iloc[-1]['最新价'] / df.iloc[-6]['最新价'] )>0.9
  return (volume_times and price_times and volume_times_31 ,
          df.iloc[-2]['开盘'], 
          df.iloc[-1]['收盘'])
  
# 个股热度前300名
query = f"个股热度前100名"
df = pywencai.get(query=query, query_type="stock",loop=True)
# print(df.columns)
# 两市剔除ST股、剔除科创板股、剔除北交所股
df = df[~df['股票代码'].astype(str).str.startswith('4')]
df = df[~df['股票代码'].astype(str).str.startswith('8')]
df = df[~df['股票代码'].astype(str).str.startswith('68')]
df = df[~df['股票简称'].astype(str).str.startswith('N')]
df = df[~df['股票简称'].astype(str).str.startswith('*')]
df = df[~df['股票简称'].astype(str).str.startswith('ST')]

df['最新价'] = df['最新价'].astype('float64')
df['最新涨跌幅'] = df['最新涨跌幅'].astype('float64')

df['代码'] = df['股票代码'].astype(str).str[:6]

df['涨跌幅大于-5'] = df['最新涨跌幅'].astype(float) > -3
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅大于-5'])
df['涨跌幅小于11'] = df['最新涨跌幅'].astype(float) < 21
df = df.where(df['涨跌幅大于-5'], np.nan).dropna(
    subset=['涨跌幅小于11'])


df['volume_up_and_price_up'] = df['代码'].apply(volume_up_and_price_up)
df['upup'] = df['volume_up_and_price_up'].map(lambda x: x[0])

df = df.where(df['upup'], np.nan).dropna(
    subset=['upup'])

df['开盘'] = df['volume_up_and_price_up'].map(lambda x: x[1])
df['开盘1分钟价格'] = df['volume_up_and_price_up'].map(lambda x: x[2])
df['开盘1分钟涨幅'] = round(100 * (df['开盘1分钟价格'] - df['开盘']) / df['开盘'],3)
# df.sort_values(by='开盘1分钟涨幅', ascending=False, inplace=True)
# df[['代码','股票简称','开盘', '开盘1分钟价格','开盘1分钟涨幅','最新价','最新涨跌幅']]

df['max_pct'] = df['股票代码'].apply(lambda code: 20 if str(code).startswith('3') else 10)  
df['qd'] = df['开盘1分钟涨幅'] / df['max_pct'] 
df.sort_values(by='qd', ascending=False, inplace=True)

df_filter = df.where(df['开盘1分钟涨幅'] <= 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘1分钟价格','开盘1分钟涨幅','最新价','最新涨跌幅']])

df_filter = df.where(df['开盘1分钟涨幅'] > 0.8 * df['最新涨跌幅']).dropna()
print(len(df),len(df_filter))
print(df_filter[['代码','股票简称','开盘', '开盘1分钟价格','开盘1分钟涨幅','最新价','最新涨跌幅']])


13 12
        代码  股票简称     开盘  开盘1分钟价格  开盘1分钟涨幅    最新价   最新涨跌幅
73  000890   法尔胜   2.50     2.70    8.000   2.70  10.204
80  300417  南华仪器   8.23     8.98    9.113   9.49  19.975
24  001208  华菱线缆   7.72     8.04    4.145   8.05   9.973
37  002165   红宝丽   3.48     3.60    3.448   3.75   9.971
38  600521  华海药业  16.68    16.97    1.739  17.79  10.019
62  002420  毅昌科技   4.78     4.85    1.464   5.31   9.938
72  002332  仙琚制药  10.70    10.85    1.402  11.72  10.047
56  002459  晶澳科技  10.35    10.42    0.676  10.49   1.060
47  600276  恒瑞医药  38.46    38.47    0.026  40.03   4.109
98  300290  荣科科技   8.68     8.52   -1.843   9.36   8.083
79  002510   天汽模   3.80     3.75   -1.316   4.09   5.959
59  600601  方正科技   2.89     2.85   -1.384   2.88  -0.690
13 1
        代码  股票简称     开盘  开盘1分钟价格  开盘1分钟涨幅    最新价  最新涨跌幅
87  600938  中国海油  35.18    35.17   -0.028  34.28 -2.281
